# FP-Kecerdasan-Buatan

## Import Library

In [1]:
import pandas as pd
import numpy as np

## Load Data Training

In [2]:
# Read fie csv
forest_df = pd.read_csv('./dataset/training.csv', header=[1])

# drop baris jika ada yang kosong
forest_df.dropna(inplace=True)

# Menghapus spasi pada header
forest_df = forest_df.rename(columns=lambda x: x.strip())

# Drop kolom hari, bulan , tahun dan index 123 karena tidak dibutuhkan
clean_df = forest_df.drop(columns=["day", "month", "year"], index=[123])
clean_df.head()

,Temperature,RH,Ws,Rain,FFMC,DMC,DC,ISI,BUI,FWI,Classes
0,29,57,18,0,65.7,3.4,7.6,1.3,3.4,0.5,not fire
1,29,61,13,1.3,64.4,4.1,7.6,1,3.9,0.4,not fire
2,26,82,22,13.1,47.1,2.5,7.1,0.3,2.7,0.1,not fire
3,25,89,13,2.5,28.6,1.3,6.9,0,1.7,0,not fire
4,27,77,16,0,64.8,3,14.2,1.2,3.9,0.5,not fire


## Attributes

In [3]:
batch_string = np.array(clean_df.drop(columns=['Classes']))

batch = []
for sample in batch_string:
    _ = []
    for attribute in sample:
        _.append(float(attribute))
    batch.append(_)

batch = np.array(batch)

print(batch)

[[29.  57.  18.  ...  1.3  3.4  0.5]
 [29.  61.  13.  ...  1.   3.9  0.4]
 [26.  82.  22.  ...  0.3  2.7  0.1]
 ...
 [27.  87.  29.  ...  0.4  3.4  0.2]
 [24.  54.  18.  ...  1.7  5.1  0.7]
 [24.  64.  15.  ...  1.2  4.8  0.5]]


## Classes

In [4]:
classes = np.array(clean_df['Classes'].str.strip())

outputs = []
for item in classes:
    if(item.strip() == 'not fire'):
        outputs.append([1])
    elif(item.strip() == 'fire'):
        outputs.append([2])

y = np.array(outputs)

print(y)

[[1]
 [1]
 [1]
 [1]
 [1]
 [2]
 [2]
 [2]
 [1]
 [1]
 [2]
 [2]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [2]
 [1]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [1]
 [2]
 [1]
 [1]
 [1]
 [1]
 [2]
 [2]
 [1]
 [2]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [2]
 [2]
 [2]
 [2]
 [2]
 [1]
 [1]
 [1]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [1]
 [1]
 [1]
 [2]
 [2]
 [2]
 [2]
 [1]
 [2]
 [2]
 [2]
 [1]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [2]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [2]
 [2]
 [2]
 [2]
 [2]
 [1]
 [1]
 [1]
 [1]
 [1]
 [2]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [2]
 [2]
 [1]
 [1]
 [2]
 [2]
 [2]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [2]
 [1]
 [1]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [1]
 [1]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [1]
 [1]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [1]
 [1]
 [1]
 [1]
 [2]
 [2]
 [2]
 [2]
 [1]
 [2]
 [2]
 [2]
 [2]
 [1]
 [1]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]


## Neural Networks

In [5]:
class Layer_Dense:
    def __init__(self, n_inputs, n_neurons):
        self.weights = np.random.randn(n_inputs, n_neurons) / 10
        self.biases = np.zeros((1, n_neurons))

    def forward(self, inputs):
        self.output = np.dot(inputs, self.weights) + self.biases


class Activation_ReLU:
    def forward(self, inputs):
        self.output = np.maximum(0, inputs)


class Activation_Softmax:
    def forward(self, inputs):
        # Eksponensial dan Prevent Overflow
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        # Normalisasi
        probabilities = exp_values / np.sum(exp_values, axis=1, keepdims=True)
        self.output = probabilities


class Loss:
    def calculate(self, output, y):
        sample_losses = self.forward(output, y)
        data_loss = np.mean(sample_losses)
        return data_loss


class Loss_CategoryCrossentropy(Loss):
    def forward(self, y_pred, y_true):
        samples = len(y_pred)
        y_pred_clipped = np.clip(y_pred, 1e-7, 1-1e-7)

        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[range(samples), y_true]

        elif len(y_true.shape) == 2:
            correct_confidences = np.sum((y_pred_clipped * y_true), axis=1)

        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods

## Training NN

In [6]:
# Hidden Layer 1
dense1 = Layer_Dense(10, 16)
activation1 = Activation_ReLU()

dense1.forward(batch)
activation1.forward(dense1.output)

# Hidden Layer 2
dense2 = Layer_Dense(16, 8)
activation2 = Activation_ReLU()

dense2.forward(activation1.output)
activation2.forward(dense2.output)

# Output Layer
dense3 = Layer_Dense(8, 2)
activation3 = Activation_Softmax()

dense3.forward(activation2.output)
activation3.forward(dense3.output)

# Loss Calculation
loss_function = Loss_CategoryCrossentropy()
loss = loss_function.calculate(activation3.output, y)

print(f"Loss : {loss}")

# Accuracy
predictions = np.argmax(batch, axis=1)
accuracy = np.mean(predictions == y)

print(f"Acc: {accuracy}")

Loss : -0.3907866820440844
Acc: 0.09514132330776134
